# The basic nbpy_top_tweeters app

First, let's get connected with the Twitter API:

In [ ]:
import os
import tweepy

In [ ]:
auth = tweepy.AppAuthHandler(
    os.environ['TWITTER_API_TOKEN'],
    os.environ['TWITTER_API_SECRET']
)

api = tweepy.API(auth)
api

In [ ]:
# import requests_cache
# requests_cache.install_cache()

At this point, we use the `search()` method to get a list of tweets matching the search term:

In [ ]:
nbpy_tweets = api.search('#nbpy', count=100)

In [ ]:
len(nbpy_tweets)

From the iterable of tweets we get the number of tweets per user by using a `collections.Counter` object:

In [ ]:
from collections import Counter

tweet_count_by_username = Counter(tweet.user.screen_name for tweet in nbpy_tweets)

In [ ]:
tweet_count_by_username

At this point, we can calculate the top $n$ tweeters:

In [ ]:
top_tweeters = tweet_count_by_username.most_common(20)
top_tweeters

And show a scoreboard with the winners:

In [ ]:
for username, tweet_count in top_tweeters:
    print(f'@{username:20}{tweet_count:2d}')

- We can see that, already with the "vanilla" notebook, we have some degree of interactivity simply by editing and running the code cell-by-cell rather than in one go

---

# From `repr()` output to rich output with `IPython.display`

In [ ]:
import random

tweet = random.choice(nbpy_tweets)
tweet

- The repr of these objects are rich in information, but not very easy to explore

In [ ]:
tweet.user

The `IPython.display` module contains several classes that render rich output from objects in a cell's output

In [ ]:
from IPython.display import *

- `JSON` turns any JSON-able `dict` into an expandable, filterable widget

In [ ]:
tweet._json

In [ ]:
JSON(tweet._json)

- `Image` generates an image from raw PNG data, a file path, or a URL

In [ ]:
Image(tweet.user.profile_image_url)

- `Markdown` can be used to generate rich text programmatically in a cell's output


In [ ]:
Markdown(f"""
*{tweet.user.name}* (`@{tweet.user.screen_name}`) is tweeting about **North Bay Python**!
""")

- `HTML` is able to render arbitrary HTML code

In [ ]:
HTML('<a class="twitter-timeline" href="https://twitter.com/northbaypython?ref_src=twsrc%5Etfw">Tweets by northbaypython</a> <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>');

- `FileLink` generates a "smart" link to a file, relative to the notebook's working directory

In [ ]:
FileLink('hey-nbpy.md')

---

We can use these building block to create rich representations and associate them with any object

- Strategy #1: Register custom formatters for object types

In [ ]:
def tweet_as_markdown(tweet):
    quoted_text = '\n'.join(f'> {line}' for line in tweet.text.split('\n'))
    author = f'--*{tweet.user.name}* (`@{tweet.user.screen_name}`) on {tweet.created_at}'
    return quoted_text + '\n\n' + author

In [ ]:
formatters = get_ipython().display_formatter.formatters
formatters['text/markdown'].for_type(tweepy.Status, tweet_as_markdown)

In [ ]:
tweet

- Strategy #2: Implement `_repr_*_()` methods for custom classes

Notes:

- Let's say we want to move the tweet-counting code to its own class


In [ ]:
class ScoreBoard:
    
    def __init__(self, items, display_top=5):
        self._items = items
        self.display_top = display_top
        
    @property
    def counts_by_name(self):
        return Counter(self._items)
    
    @property
    def to_display(self):
        return self.counts_by_name.most_common(self.display_top)

    def _repr_markdown_(self):
        # effectively we're using this 
        lines = [
            f'# [North Bay Python 2019](https://2019.northbaypython.org) Top {self.display_top} Tweeters',
            '| name | # tweets |',
            '|-|-|',
        ]

        for name, count in self.to_display:
            lines.append(f'| {name} | {count} |')

        return '\n'.join(lines)
        

In [ ]:
ScoreBoard(tweet_count_by_username, display_top=10)

- Rich output is rendered automatically when the object is the return value of a cell
    - Tip: use a `;` at the end of the last line in the cell to render nothing instead
- Use the `display()` function to show rich output from anywhere in a cell (e.g. in a loop)
    - `display()` is versatile; falls back to text repr in a console

In [ ]:
for tweet in nbpy_tweets[:10]:
    display(tweet)

In [ ]:
nbpy_tweets = api.search('#nbpy', count=100000)

In [ ]:
import pickle

with open('nbpy_tweets.pkl', 'wb') as f:
    pickle.dump(nbpy_tweets, f)

In [ ]:
with open('nbpy_tweets.pkl', 'rb') as f:
    unpickled = pickle.load(f)
    
len(unpickled)